In [2]:
# Import all packages used
import pandas as pd
import numpy as np
import spacy as sp
import string
import en_core_web_sm
import nltk as nl
import re

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

from sklearn import svm

from sklearn.ensemble import RandomForestClassifier


from sklearn.naive_bayes import BernoulliNB

model = svm.SVC()
param = {'kernel': ['linear','rbf']}

lemm_obj = sp.load('en_core_web_lg')
lemm_obj.max_length = 1500000
ps = nl.PorterStemmer()

vect_cnt = CountVectorizer(analyzer='word',min_df=1,stop_words='english',token_pattern='[a-zA-Z0-9]{1,}')
tfidf_vect = TfidfVectorizer(analyzer = 'word', min_df = 1, stop_words = 'english',vocabulary=20000, token_pattern = '[a-zA-Z0-9]{1,}')

allowed_postags = ['NOUN','ADJ','VERB','ADV']

import pandas as pd
from collections import deque
import random
import numpy as np
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, LSTM, CuDNNLSTM, BatchNormalization, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import time
from sklearn import preprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
import IPython

C:\Users\ragha\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ragha\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ragha\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ragha\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [3]:
trailerData = pd.read_pickle(r"D:\Machine Learning\Vamsi Thesis\Code\trailerTranscripts.pkl")
trailerData = trailerData[trailerData['trailerTranscript']!='']
trailerData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23112 entries, 0 to 25434
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   youtubeId          23112 non-null  object
 1   movieId            23112 non-null  int64 
 2   title              23112 non-null  object
 3   trailerTranscript  23112 non-null  object
dtypes: int64(1), object(3)
memory usage: 902.8+ KB


In [5]:
movieId = pd.read_pickle(r"D:\Machine Learning\Vamsi Thesis\Code\moviePlots.pkl")
plotData = movieId[movieId['plot']!='']
plotData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42457 entries, 0 to 62421
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  42457 non-null  int64  
 1   imdbId   42457 non-null  int64  
 2   tmdbId   42362 non-null  float64
 3   plot     42457 non-null  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 1.6+ MB


In [6]:
plotTrailer = trailerData[['movieId','title','trailerTranscript']].merge(plotData[['movieId','plot']], on='movieId',how='inner')
plotTrailer.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12844 entries, 0 to 12843
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   movieId            12844 non-null  int64 
 1   title              12844 non-null  object
 2   trailerTranscript  12844 non-null  object
 3   plot               12844 non-null  object
dtypes: int64(1), object(3)
memory usage: 501.7+ KB


In [20]:
genreData = pd.read_csv(r"D:\Machine Learning\Vamsi Thesis\Data\ml-25m\movies.csv",usecols=["movieId","genres"])
genreData.info()
fullMovieData = plotTrailer.merge(genreData,on='movieId',how='inner')
fullMovieData
# Removing dummy plots
test = fullMovieData[fullMovieData['movieId']==71]
plot = (test.loc[:,'plot']).tolist()
fullMovieData =  fullMovieData[fullMovieData['plot']!=plot[0]]
fullMovieData = fullMovieData[fullMovieData['genres']!='(no genres listed)']
fullMovieData['ttLength']=fullMovieData.apply(lambda x: len(x['trailerTranscript']),axis=1)
fullMovieData=fullMovieData[fullMovieData['ttLength']<=10000]
fullMovieData=fullMovieData[fullMovieData['ttLength']>=100]
fullMovieData.describe(percentiles=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.95,0.99,1])
fullMovieData
fullMovieData.reset_index(inplace=True)
fullMovieData.to_pickle(r'D:\Machine Learning\Vamsi Thesis\Code\Results\fullMovieData.pkl')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  62423 non-null  int64 
 1   genres   62423 non-null  object
dtypes: int64(1), object(1)
memory usage: 975.5+ KB


In [8]:
# List of genres
g_list = fullMovieData.apply(lambda x: x['genres'].split('|'),axis = 1)
g_list=g_list.to_list()
g_list=list(set(sum(g_list,[])))
g_list

['Adventure',
 'Romance',
 'Horror',
 'Comedy',
 'Mystery',
 'Action',
 'Film-Noir',
 'Thriller',
 'Drama',
 'IMAX',
 'Fantasy',
 'War',
 'Animation',
 'Crime',
 'Western',
 'Sci-Fi',
 'Musical',
 'Children',
 'Documentary']

In [9]:
#Genre one hot encoding
data = np.zeros(shape=(len(fullMovieData),len(g_list)+1))
gen_df_encode = pd.DataFrame(data,columns=['movieId']+g_list)
gen_df_encode = pd.concat([fullMovieData,gen_df_encode],axis=1)
gen_df_encode

,index,movieId,title,trailerTranscript,plot,genres,ttLength,movieId,Adventure,Romance,...,IMAX,Fantasy,War,Animation,Crime,Western,Sci-Fi,Musical,Children,Documentary
0,0,1,Toy Story (1995),sergeant yes sir establish a recon post downst...,A little boy named Andy loves to be in his roo...,Adventure|Animation|Children|Comedy|Fantasy,676,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2,Jumanji (1995),It's gotta be micro-chipped or something You g...,"Jumanji, one of the most unique--and dangerous...",Adventure|Children|Fantasy,249,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,3,Grumpier Old Men (1995),or on hot crankier my dog was as ugly as you l...,Things don't seem to change much in Wabasha Co...,Comedy|Romance,1053,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,4,Waiting to Exhale (1995),[Music] savannah bernadine robin gloria [Music...,This story based on the best selling novel by ...,Comedy|Drama|Romance,1740,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,5,Father of the Bride Part II (1995),[Music] we're George banks life was good it wa...,"In this sequel to ""Father of the Bride"", Georg...",Comedy,1499,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10397,12839,124273,Kevin Smith: Too Fat For 40 (2010),"I was just wondering,\nin all the Cousin Walte...",A no holds barred concert with award winning f...,Comedy,8300,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10398,12840,125429,Johnny Skidmarks (1998),[Music] [Applause] [Music] [Applause] [Music] ...,"Crime scene photographer, Johnny Scardino (aka...",Crime|Drama|Mystery|Thriller,3113,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10399,12841,127272,Nobody Knows Anything! (2003),I came into this business when I was at Prince...,"In this comedy about Hollywood, Sarah thinks m...",Comedy,636,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10400,12842,128439,Point and Shoot (2014),I was raised on action movies all my adventure...,Matthew VanDyke presents slices of his life fr...,Documentary,1547,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
def genre_one_hot_encoding(df):
    for i in range(0, len(df)):
        genres = str(df.loc[i, 'genres'])
        genres = genres.strip()
        genres = genres.split('|')
        #print(genres)
        for g in genres:
            df.loc[i,g] = int(1)
    return df
gen_df_encode = genre_one_hot_encoding(gen_df_encode)
print(gen_df_encode.shape)
gen_df_encode.head()

(10402, 27)


,index,movieId,title,trailerTranscript,plot,genres,ttLength,movieId,Adventure,Romance,...,IMAX,Fantasy,War,Animation,Crime,Western,Sci-Fi,Musical,Children,Documentary
0,0,1,Toy Story (1995),sergeant yes sir establish a recon post downst...,A little boy named Andy loves to be in his roo...,Adventure|Animation|Children|Comedy|Fantasy,676,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,2,Jumanji (1995),It's gotta be micro-chipped or something You g...,"Jumanji, one of the most unique--and dangerous...",Adventure|Children|Fantasy,249,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2,3,Grumpier Old Men (1995),or on hot crankier my dog was as ugly as you l...,Things don't seem to change much in Wabasha Co...,Comedy|Romance,1053,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,4,Waiting to Exhale (1995),[Music] savannah bernadine robin gloria [Music...,This story based on the best selling novel by ...,Comedy|Drama|Romance,1740,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,5,Father of the Bride Part II (1995),[Music] we're George banks life was good it wa...,"In this sequel to ""Father of the Bride"", Georg...",Comedy,1499,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
#Text Processing
def stemming(text):
    text = ''.join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('[^a-zA-Z]*', text)
    text = [ps.stem(word) for word in tokens]
    return ' '.join(text)
def Lemm(text):
    texts_out = []
    text = ''.join([word.lower() for word in text if word not in string.punctuation])
    doc = lemm_obj(text)
    x=' '.join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc])
    return re.sub(' +',' ',x)

In [12]:
import time 
start = time.time()
# gen_df_encode['tt_stemmed'] = gen_df_encode.apply(lambda x: stemming(x['trailerTranscript']),axis = 1)
gen_df_encode['tt_lemm'] = gen_df_encode.apply(lambda x: Lemm(x['trailerTranscript']),axis = 1)
# gen_df_encode['plot_stemmed'] = gen_df_encode.apply(lambda x: stemming(x['plot']),axis = 1)
gen_df_encode['plot_lemm'] = gen_df_encode.apply(lambda x: Lemm(x['plot']),axis = 1)
gen_df_encode

end = time.time()
total_time = end-start

print("Total Time taken:" + str(end-start))


Total Time taken:513.5435507297516


In [19]:
gen_df_encode.to_pickle(r'D:\Machine Learning\Vamsi Thesis\Code\Results\gen_df_encode.pkl')

In [13]:
Final_data=pd.DataFrame(columns=["Genre","Actual Positives","Actual Negatives","Vectorizer","Model","Test_Precision","Test_Recall","Test_Fscore","Precision","Recall","Fscore"])
Final_data.shape
#Master function SVM RF Naive Bayes
def prediction(df,text, genre, vectorizer, model, param):
    result=[]
    print("Genre for prediction: ", genre)
    result.append(genre)
    
    #Sampling
    df_model_0 = df[df[genre] == 0]
    df_model_1 = df[df[genre] == 1]
    print("Number of 0s: ", df_model_0.shape[0])
    print("Number of 1s: ", df_model_1.shape[0])
    result.append(df_model_0.shape[0])
    result.append(df_model_1.shape[0]) 
    
    sample_size = 2*df_model_1.shape[0]
    if sample_size > len(df_model_0) : sample_size = len(df_model_0)
    df_model_s=pd.concat([df_model_1,df_model_0.sample(sample_size)])
    print("Shape of the Sampled DataFrame: ", df_model_s.shape)
    #result.append(df_model_s.shape[0])
    
    #Train-Test Split
    X_df = df_model_s[text]
    y_df = df_model_s[genre]
    X_train,X_test,y_train,y_test = train_test_split(X_df, y_df, test_size = 0.25, random_state = 1,
                                                     shuffle = True)
    
    #Vectorization
    print("\n")
    print("Vectorization Start")
    vect = vectorizer(analyzer = 'word', min_df = 1, stop_words = 'english', token_pattern = '[a-zA-Z]{2,}')
    vect_fit = vect.fit(X_train)
    result.append(vectorizer)
    
    X_train_vec = pd.DataFrame(vect_fit.transform(X_train).toarray())
    X_train_vec.columns = vect_fit.get_feature_names()
    
    X_test_vec = pd.DataFrame(vect_fit.transform(X_test).toarray())
    X_test_vec.columns = vect_fit.get_feature_names()
    print("Vectorization End")
    
    #Model
    print("\n")
    print("Model Building Start")
    model = model
    param = param
    result.append(model)
    
    gs = GridSearchCV(model, param, cv = 5,n_jobs=2)
    gs_fit = gs.fit(X_train_vec, y_train)
    
    test_pred_cnt = pd.DataFrame(gs.predict(X_test_vec))
    train_pred_cnt = pd.DataFrame(gs.predict(X_train_vec))
    print("Model Building End")

    # Accuracy metrics

    #Test Accuracy
    print("\n")
    print("Sample data Test Set Confusion Matrix: ")
    print(confusion_matrix(y_test, test_pred_cnt))
    precision, recall, fscore, train_support = score(y_test, test_pred_cnt, pos_label = 1, average = 'binary')
    print("Sample data Test Set Precision: ", precision)
    print("Sample data Test Set Recall: ", recall)
    print("Sample data Test Set FScore: ", fscore)
    print("\n")
    result.extend([precision, recall, fscore])

    # Train Accuracy
    print("Sample data Train Set Confusion Matrix: ")
    print(confusion_matrix(y_train, train_pred_cnt))
    precision, recall, fscore, train_support = score(y_train, train_pred_cnt, pos_label = 1, average = 'binary')
    print("Sample data Train Set (Precision, Recall, Fscore) :",precision, recall, fscore)
    print("\n")

    #Accuracy on full Data
    X_df_full = df[text]
    y_df_full = df[genre]

    # vectorizer
    X_full_tf = pd.DataFrame(vect_fit.transform(X_df_full).toarray())

    print("Probability Evaluation and Concat Start")
    full_pred_cnt = pd.DataFrame(gs.predict(X_full_tf))
    full_pred_prob = pd.DataFrame(gs.predict_proba(X_full_tf))
    df = pd.concat([df,full_pred_prob], axis = 1)
    df.rename(columns ={1:genre+'_prob'},inplace=True)
    df.drop([0],axis=1,inplace=True)
    print("Probability Evaluation and Concat End")
    
    print("\n")
    print("Full Dataset Confusion Matrix " , model)
    print(confusion_matrix(y_df_full, full_pred_cnt))
    precision, recall, fscore, train_support = score(y_df_full, full_pred_cnt, pos_label = 1, average = 'binary')
    print("Full Dataset Precision: ", precision)
    print("Full Dataset Recall: ", recall)
    print("Full Dataset FScore: ", fscore)
    print("\n")
    result.extend([precision, recall, fscore])
    
    print("Appending results to Final_data Start")
    Final_data.loc[len(Final_data)]=result
    print("Appending results to Final_data End")
    return df

In [14]:
start = time.time()
df_test=gen_df_encode
df_test= prediction(df_test,'plot_lemm','Comedy',TfidfVectorizer, RandomForestClassifier(), {'n_estimators': [10,25,50,100],'max_depth':[100]})
end = time.time()
total_time = end-start

print("Total Time taken:" + str(end-start))

Genre for prediction:  Comedy
Number of 0s:  6914
Number of 1s:  3488
Shape of the Sampled DataFrame:  (10402, 29)


Vectorization Start
Vectorization End


Model Building Start
Model Building End


Sample data Test Set Confusion Matrix: 
[[1683   32]
 [ 751  135]]
Sample data Test Set Precision:  0.8083832335329342
Sample data Test Set Recall:  0.1523702031602709
Sample data Test Set FScore:  0.25641025641025644


Sample data Train Set Confusion Matrix: 
[[5199    0]
 [ 201 2401]]
Sample data Train Set (Precision, Recall, Fscore) : 1.0 0.9227517294388932 0.959824105536678


Probability Evaluation and Concat Start
Probability Evaluation and Concat End


Full Dataset Confusion Matrix  RandomForestClassifier()
[[6882   32]
 [ 952 2536]]
Full Dataset Precision:  0.9875389408099688
Full Dataset Recall:  0.7270642201834863
Full Dataset FScore:  0.8375165125495377


Appending results to Final_data Start
Appending results to Final_data End
Total Time taken:563.3733780384064


In [15]:
start = time.time()
df_test=gen_df_encode
for i in g_list:    
    df_test= prediction(df_test,'plot_lemm',i,TfidfVectorizer, RandomForestClassifier(), {'n_estimators': [10,25,50,100],'max_depth':[100]})
df_test.to_pickle(r'D:\Machine Learning\Vamsi Thesis\Code\Results\RF_Probability_Distribution_plots.pkl',index=False)
total_time = end-start

print("Total Time taken:" + str(end-start))

Genre for prediction:  Adventure
Number of 0s:  9516
Number of 1s:  886
Shape of the Sampled DataFrame:  (2658, 29)


Vectorization Start
Vectorization End


Model Building Start
Model Building End


Sample data Test Set Confusion Matrix: 
[[421  25]
 [130  89]]
Sample data Test Set Precision:  0.7807017543859649
Sample data Test Set Recall:  0.4063926940639269
Sample data Test Set FScore:  0.5345345345345345


Sample data Train Set Confusion Matrix: 
[[1326    0]
 [  12  655]]
Sample data Train Set (Precision, Recall, Fscore) : 1.0 0.9820089955022488 0.9909228441754917


Probability Evaluation and Concat Start
Probability Evaluation and Concat End


Full Dataset Confusion Matrix  RandomForestClassifier()
[[9079  437]
 [ 142  744]]
Full Dataset Precision:  0.6299745977984759
Full Dataset Recall:  0.8397291196388262
Full Dataset FScore:  0.7198838896952106


Appending results to Final_data Start
Appending results to Final_data End
Genre for prediction:  Romance
Number of 0s:  8728
Numbe

C:\Users\ragha\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Sample data Test Set Precision:  0.0
Sample data Test Set Recall:  0.0
Sample data Test Set FScore:  0.0


Sample data Train Set Confusion Matrix: 
[[37  0]
 [ 0 26]]
Sample data Train Set (Precision, Recall, Fscore) : 1.0 1.0 1.0


Probability Evaluation and Concat Start
Probability Evaluation and Concat End


Full Dataset Confusion Matrix  RandomForestClassifier()
[[10247   127]
 [    2    26]]
Full Dataset Precision:  0.16993464052287582
Full Dataset Recall:  0.9285714285714286
Full Dataset FScore:  0.28729281767955805


Appending results to Final_data Start
Appending results to Final_data End
Genre for prediction:  Thriller
Number of 0s:  8336
Number of 1s:  2066
Shape of the Sampled DataFrame:  (6198, 36)


Vectorization Start
Vectorization End


Model Building Start
Model Building End


Sample data Test Set Confusion Matrix: 
[[988  60]
 [313 189]]
Sample data Test Set Precision:  0.7590361445783133
Sample data Test Set Recall:  0.37649402390438247
Sample data Test Set FScore:  0

Vectorization Start
Vectorization End


Model Building Start
Model Building End


Sample data Test Set Confusion Matrix: 
[[309  16]
 [ 61 100]]
Sample data Test Set Precision:  0.8620689655172413
Sample data Test Set Recall:  0.6211180124223602
Sample data Test Set FScore:  0.7220216606498194


Sample data Train Set Confusion Matrix: 
[[971   0]
 [  2 485]]
Sample data Train Set (Precision, Recall, Fscore) : 1.0 0.9958932238193019 0.9979423868312758


Probability Evaluation and Concat Start
Probability Evaluation and Concat End


Full Dataset Confusion Matrix  RandomForestClassifier()
[[9345  409]
 [  63  585]]
Full Dataset Precision:  0.5885311871227364
Full Dataset Recall:  0.9027777777777778
Full Dataset FScore:  0.7125456760048722


Appending results to Final_data Start
Appending results to Final_data End
Genre for prediction:  Musical
Number of 0s:  10118
Number of 1s:  284
Shape of the Sampled DataFrame:  (852, 45)


Vectorization Start
Vectorization End


Model Building Start
M

TypeError: to_pickle() got an unexpected keyword argument 'index'

In [17]:
df_test.to_pickle(r'D:\Machine Learning\Vamsi Thesis\Code\Results\RF_Probability_Distribution_plots.pkl')

In [ ]:
df_test=gen_df_encode
for i in g_list:    
    df_test = prediction(df_test,'plot_lemm',i,TfidfVectorizer, svm.SVC(probability=True),  {'kernel': ['linear','rbf']})
df_test.to_pickle(r'D:\Machine Learning\Vamsi Thesis\Code\Results\SVM_Probability_Distribution_plots.pkl',index=False)

Genre for prediction:  Comedy
Number of 0s:  6914
Number of 1s:  3488
Shape of the Sampled DataFrame:  (10402, 29)


Vectorization Start
Vectorization End


Model Building Start
